In [1]:
import pandas as pd
import numpy as np

In [2]:
gen1 = pd.read_csv(".././Data Sets/Plant_1_Generation_Data.csv")
weather1 = pd.read_csv(".././Data Sets/Plant_1_Weather_Sensor_Data.csv")

gen2 = pd.read_csv(".././Data Sets/Plant_2_Generation_Data.csv")
weather2 = pd.read_csv(".././Data Sets/Plant_2_Weather_Sensor_Data.csv")


- Convert DATE_TIME Columns to Datetime

In [3]:
for df in [gen1, weather1, gen2, weather2]:
    df['DATE_TIME'] = pd.to_datetime(df['DATE_TIME'], errors='coerce', dayfirst=True)


C:\Users\peerm\AppData\Local\Temp\ipykernel_11304\4063308798.py:2: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['DATE_TIME'] = pd.to_datetime(df['DATE_TIME'], errors='coerce', dayfirst=True)
C:\Users\peerm\AppData\Local\Temp\ipykernel_11304\4063308798.py:2: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['DATE_TIME'] = pd.to_datetime(df['DATE_TIME'], errors='coerce', dayfirst=True)
C:\Users\peerm\AppData\Local\Temp\ipykernel_11304\4063308798.py:2: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['DATE_TIME'] = pd.to_datetime(df['DATE_TIME'], errors='coerce', dayfirst=True)


In [4]:
for name, df in zip(['gen1', 'weather1', 'gen2', 'weather2'], [gen1, weather1, gen2, weather2]):
    print(name, df['DATE_TIME'].isna().sum())

gen1 0
weather1 0
gen2 0
weather2 0


In [5]:

print("Shapes:")
print("gen1:", gen1.shape)
print("weather1:", weather1.shape)
print("gen2:", gen2.shape)
print("weather2:", weather2.shape)

Shapes:
gen1: (68778, 7)
weather1: (3182, 6)
gen2: (67698, 7)
weather2: (3259, 6)


In [6]:
# Step 3: Clean column names and inspect key columns

for df in [gen1, weather1, gen2, weather2]:
    df.columns = df.columns.str.strip()  # remove extra spaces

print("Plant 1 generation columns:", gen1.columns.tolist())
print("Plant 1 weather columns:", weather1.columns.tolist())
print("Plant 2 generation columns:", gen2.columns.tolist())
print("Plant 2 weather columns:", weather2.columns.tolist())

Plant 1 generation columns: ['DATE_TIME', 'PLANT_ID', 'SOURCE_KEY', 'DC_POWER', 'AC_POWER', 'DAILY_YIELD', 'TOTAL_YIELD']
Plant 1 weather columns: ['DATE_TIME', 'PLANT_ID', 'SOURCE_KEY', 'AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE', 'IRRADIATION']
Plant 2 generation columns: ['DATE_TIME', 'PLANT_ID', 'SOURCE_KEY', 'DC_POWER', 'AC_POWER', 'DAILY_YIELD', 'TOTAL_YIELD']
Plant 2 weather columns: ['DATE_TIME', 'PLANT_ID', 'SOURCE_KEY', 'AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE', 'IRRADIATION']


In [7]:
# Step : Drop missing or invalid rows before merging

# Drop rows where DATE_TIME is missing
gen1.dropna(subset=['DATE_TIME'], inplace=True)
weather1.dropna(subset=['DATE_TIME'], inplace=True)
gen2.dropna(subset=['DATE_TIME'], inplace=True)
weather2.dropna(subset=['DATE_TIME'], inplace=True)

# Drop any remaining NaNs just in case
for df in [gen1, weather1, gen2, weather2]:
    df.dropna(inplace=True)

print("After cleaning:")
print("gen1:", gen1.shape)
print("weather1:", weather1.shape)
print("gen2:", gen2.shape)
print("weather2:", weather2.shape)


After cleaning:
gen1: (68778, 7)
weather1: (3182, 6)
gen2: (67698, 7)
weather2: (3259, 6)


In [8]:
# Step : Merge generation and weather data (using nearest timestamp)
def merge_nearest(gen, weather):
    # Sort both by datetime
    gen = gen.sort_values('DATE_TIME')
    weather = weather.sort_values('DATE_TIME')
    
    # Merge based on the nearest timestamp (within 10 minutes)
    merged = pd.merge_asof(
        gen, weather,
        on='DATE_TIME',
        by='PLANT_ID',
        tolerance=pd.Timedelta('10min'),
        direction='nearest'
    )
    return merged

plant1 = merge_nearest(gen1, weather1)
plant2 = merge_nearest(gen2, weather2)

print("✅ Merged successfully using nearest timestamps!")
print("Plant 1 shape:", plant1.shape)
print("Plant 2 shape:", plant2.shape)


✅ Merged successfully using nearest timestamps!
Plant 1 shape: (68778, 11)
Plant 2 shape: (67698, 11)


In [9]:
# Step : Add time features (hour, day, month)

for df in [plant1, plant2]:
    df['HOUR'] = df['DATE_TIME'].dt.hour
    df['DAY'] = df['DATE_TIME'].dt.day
    df['MONTH'] = df['DATE_TIME'].dt.month

print("Added time features!")
print("Sample columns:", plant1.columns.tolist())

Added time features!
Sample columns: ['DATE_TIME', 'PLANT_ID', 'SOURCE_KEY_x', 'DC_POWER', 'AC_POWER', 'DAILY_YIELD', 'TOTAL_YIELD', 'SOURCE_KEY_y', 'AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE', 'IRRADIATION', 'HOUR', 'DAY', 'MONTH']


In [10]:
# Step : Clean and prepare final data for model training

# Drop unwanted or duplicate columns
for df in [plant1, plant2]:
    for col in ['PLANT_ID_x', 'PLANT_ID_y']:
        if col in df.columns:
            df.drop(columns=col, inplace=True)
    df.drop_duplicates(inplace=True)

# Combine both plants into one dataset
combined = pd.concat([plant1, plant2], ignore_index=True)

# Drop rows where key features are missing
combined.dropna(subset=['DC_POWER', 'AMBIENT_TEMPERATURE',
                        'MODULE_TEMPERATURE', 'IRRADIATION'], inplace=True)

# Filter out rows with IRRADIATION = 0 (nighttime — no solar output)
combined = combined[combined['IRRADIATION'] > 0]

# Step : Filter out rows with zero DC power
before = combined.shape[0]
combined = combined[combined['DC_POWER'] > 0]
after = combined.shape[0]
print(f"Filtered out {before - after} zero-power rows. Remaining: {after}")

# Drop unnecessary columns
combined.drop(columns=['PLANT_ID', 'SOURCE_KEY_x', 'SOURCE_KEY_y', 'DAY'], inplace=True)

# Convert date to datetime
combined['DATE_TIME'] = pd.to_datetime(combined['DATE_TIME'])


# Define features and target
features = ['AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE', 'IRRADIATION', 'HOUR', 'MONTH']
X = combined[features]
y = combined['DC_POWER']

print("✅ Data ready for model training!")
print("X shape:", X.shape)
print("y shape:", y.shape)
print("\nSample X rows:")
print(X.head())
print("\nSample y rows:")
print(y.head())

# 💾 Step 8: Save cleaned dataset for reuse
combined.to_csv(".././Data Sets/Cleaned_Solar_Data.csv", index=False)
print("\n✅ Cleaned dataset saved successfully as 'Cleaned_Solar_Data.csv'")


Filtered out 8253 zero-power rows. Remaining: 68845
✅ Data ready for model training!
X shape: (68845, 5)
y shape: (68845,)

Sample X rows:
     AMBIENT_TEMPERATURE  MODULE_TEMPERATURE  IRRADIATION  HOUR  MONTH
510            24.088446           22.206757     0.005887     6      5
511            24.088446           22.206757     0.005887     6      5
512            24.088446           22.206757     0.005887     6      5
513            24.088446           22.206757     0.005887     6      5
514            24.088446           22.206757     0.005887     6      5

Sample y rows:
510    41.857143
511    57.000000
512    54.625000
513    54.625000
514    38.625000
Name: DC_POWER, dtype: float64

✅ Cleaned dataset saved successfully as 'Cleaned_Solar_Data.csv'


In [11]:
print("Combined shape:", combined.shape)


Combined shape: (68845, 10)
